In [5]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
df = pd.read_csv("data/all_data.csv")

C:\Users\thbom\AppData\Local\Temp\ipykernel_25332\3684764311.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/all_data.csv")


In [7]:
df

,Unnamed: 0,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,Unnamed: 62,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 42,Unnamed: 43,Unnamed: 49,Unnamed: 50
0,0,F1,2004/8/6,NaN,Nice,Lyon,0.0,1.0,A,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,D1,2004/8/6,NaN,Werder Bremen,Schalke 04,1.0,0.0,H,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,F1,2004/8/7,NaN,Bastia,Strasbourg,2.0,1.0,H,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,F1,2004/8/7,NaN,Caen,Istres,1.0,1.0,D,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,D1,2004/8/7,NaN,Dortmund,Wolfsburg,1.0,2.0,A,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38765,38765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38766,38766,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38767,38767,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38768,38768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# remove unknown columns
columns = df.columns
unknown_cols = [c for c in columns if "Unnamed" in c]
df = df.drop(columns=unknown_cols)

# remove rows with only na values
df = df.dropna(how="all")

features_to_keep = ["Div", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HS", "AS", "HST", "AST", 
                    "B365H", "B365D", "B365A",
                    "BWH", "BWD", "BWA",
                    "AHh", "B365AHH", "B365AHA",
                    "AHCh", "B365CAHH", "B365CAHA",
                    "B365>2.5", "B365<2.5",
                    ]
df = df[features_to_keep]

In [9]:
name_conversions = {
'St Etienne': 'Saint-Etienne',
'Ajaccio GFCO': 'Ajaccio', 
'Ath Madrid': 'Atletico',
'Ein Frankfurt': 'Frankfurt',
'Espanol': 'Espanyol',
'La Coruna': 'Depor',
'Nurnberg': 'Nuernberg',
'M\'gladbach': 'Gladbach',
'Bayern Munich': 'Bayern',
'Greuther Furth': 'Fuerth',
'Sp Gijon': 'Gijon',
'FC Koln': 'Koeln',
'Ath Bilbao': 'Bilbao',
'Hansa Rostock': 'Rostock',
'Fortuna Dusseldorf': 'Duesseldorf',
'Schalke 04': 'Schalke',
'Werder Bremen': 'Werder',
'Vallecano': 'RayoVallecano',
'Kaiserslautern': 'Lautern',
"Nott'm Forest": 'Forest',
'Holstein Kiel': 'Holstein',
'Munich 1860': 'Munich'
}

In [10]:
from os.path import exists
all_teams = set(df["HomeTeam"].unique()).union(set(df["AwayTeam"].unique()))


In [11]:

club_elo_base_url = "http://api.clubelo.com/"
name_mismatches = []
exceptions = []
for team_name in all_teams:
    try:
        # if exists("elo/" + team_name + ".csv"):
        #     continue
        team_name_url = name_conversions.get(team_name, team_name).replace(" ", "")
        team_data = requests.get(club_elo_base_url + team_name_url, stream=True)
        byte_data = team_data.content
        if team_name == "Schalke 04":
            a = 1
        if len(byte_data) == 38 or team_data.status_code != 200:
            # means that only header was returned, so team was not found
            name_mismatches.append(team_name)
            continue
        with open("elo/" + team_name + ".csv", mode='wb') as f:
            f.write(team_data.content)
    except:
        exceptions.append(team_name)
        continue
print("Name mismatches: ", name_mismatches)
print("Exceptions: ", exceptions)

Name mismatches:  ['Gimnastic', 'Evian Thonon Gaillard', 'Arles', 'Carpi', 'Nimes']
Exceptions:  ['Swansea', 'Parma', 'Schalke 04', 'Sedan', 'Leverkusen', 'Le Havre', 'Lens', 'St Etienne', 'Watford', 'Nancy', 'Greuther Furth', 'Man City']


In [ ]:
import datetime
from functools import partial

# for team in all_teams:
#     df.loc[(df["HomeTeam"] == team) | (df["AwayTeam"] == team)]
# # df = df.apply(add_elo_feature, axis=1)
# df.to_csv("data/all_data_with_elo.csv")
# df

dfs = {}

for team in all_teams:
    if exists("elo/" + team + ".csv"):
        dfs[team] = pd.read_csv("elo/" + team + ".csv")
        
def find_team_elo_at(team_name, date):
    elo_df = dfs.get(team_name, None)
    if elo_df is None:
        return None
    elo_df["From"] = pd.to_datetime(elo_df["From"], format='mixed', dayfirst=True)
    elo_df["To"] = pd.to_datetime(elo_df["To"], format='mixed', dayfirst=True)
    if date > '2000-05-01':
        aa = 1
    # if date > '2024-10-01':
    #     aa = 2
    try:
        # elo = elo_df.loc[elo_df["From"] >= date]["Elo"].iloc[0]
        # print(elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] > date)])
        elo = elo_df.loc[(elo_df["From"] <= date) & (elo_df["To"] >= date)]["Elo"].iloc[0]
        return elo
    except:
        return None
    
def add_elo_feature(s):
    s["HomeTeamELO"] = find_team_elo_at(s["HomeTeam"], s["Date"])
    s["AwayTeamELO"] = find_team_elo_at(s["AwayTeam"], s["Date"])
    return s

f = np.vectorize(find_team_elo_at)

df["HomeTeamELO"] = f(df["HomeTeam"], df["Date"])
df["AwayTeamELO"] = f(df["AwayTeam"], df["Date"])

# df = df.apply(add_elo_feature, axis=1)
df.to_csv("data/all_data_with_elo.csv")
df

In [ ]:
df["Date"][10000] > '2024-10-01'